In [15]:
import pyodbc
import pandas as pd 
import requests
import geocoder
from geopy.geocoders import Nominatim
from IPython.display import clear_output
from pprint import pprint
import time
import re
import timeit
from IPython.display import clear_output

In [56]:
cnxn = pyodbc.connect('DSN=SQLAmlin;UID=MSAmlin;PWD=password')

In [57]:
cur = cnxn.cursor()

### Define function for getting address parts


In [4]:
def get_full_address(part_addr):
    '''Takes an address and returns a dictionary containing the components of the address.
    Works best with a road name and town'''
    time.sleep(1)
    # This outer try/except block ensures that a dictionary of the 
    # correct form is returned if the geocoder bit doesn't work
    # so that key errors won't stop this from being applied to a dataframe column
    try:
        addr_dict = {}
        
        g = geocoder.osm(part_addr)
        try:
            addr_dict['HouseNumber'] = g.json['raw']['address']['house_number'] 
        except KeyError:
            addr_dict['HouseNumber'] = None
        
        try:
            addr_dict['Road'] = g.json['raw']['address']['road']
        except KeyError:
            addr_dict['Road'] = None
        
        # The 'town' bit of the address is sometimes stored under different keys in the json
        try:
            addr_dict['Town'] = g.json['raw']['address']['town']
        except KeyError:
            try:
                addr_dict['Town'] = g.json['raw']['address']['suburb']
            except KeyError:
                try:
                    addr_dict['Town'] = g.json['raw']['address']['city']
                except:
                    addr_dict['Town'] = None
        
        try:
            addr_dict['Postcode'] = g.json['raw']['address']['postcode']
        except KeyError:
            addr_dict['Postcode'] = None
            
        try:
            addr_dict['Lat'] = g.json['lat']
        except KeyError:
            addr_dict['Lat'] = None
        
        try:
            addr_dict['Long'] = g.json['lng']
        except KeyError:
            addr_dict['Long'] = None
        
        return addr_dict    
    
    # If geocoder lookup doesn't work then return a dictionary of 'location not found' to differentiate between the None values 
    except:
        return {'HouseNumber': 'Location not found', 'Road': 'Location not found', 'Town': 'Location not found', 'Postcode': 'Location not found', 'Lat': 'Location not found', 'Long': 'Location not found'}

    
    

### Cambridgeshire Geocoding

In [5]:
cambs_query = 'select * from initial.CambridgeshireFire'

In [6]:
cambs_df = pd.read_sql(cambs_query, cnxn)

In [7]:
# get rid of weird characters
cambs_df['title'] = cambs_df['title'].map(lambda x: x.replace('â€“', '-'))
# incident type stored with address - split on ' - ' to separate, then drop the first list item, then join to connect elements of address
cambs_df['location'] = cambs_df['title'].map(lambda x: ', '.join(x.split(' - ')[1:]))
# when locations have only 1 word, put cambridgeshire in the address to increase chance of OSM finding the right one
cambs_df['location'] = cambs_df['location'].map(lambda x: x + ', Cambridgeshire' if (x.count(', ') == 0)  else x)


In [18]:
# Geocoding using get_full_address function run on the cleaned location column

cambs_df['AddressDict'] = cambs_df['location'].map(lambda x: get_full_address(x))

cambs_df['HouseNumber'] = cambs_df['AddressDict'].map(lambda x: x['HouseNumber'])
cambs_df['Road'] = cambs_df['AddressDict'].map(lambda x: x['Road'])
cambs_df['Town'] = cambs_df['AddressDict'].map(lambda x: x['Town'])
cambs_df['Postcode'] = cambs_df['AddressDict'].map(lambda x: x['Postcode'])
cambs_df['Lat'] = cambs_df['AddressDict'].map(lambda x: x['Lat'])
cambs_df['Long'] = cambs_df['AddressDict'].map(lambda x: x['Long'])

In [27]:
# No postcodes or LatLongs were in the original data so all flagged as inferred
cambs_df['PostcodeInferredFlag'] = 1
cambs_df['LatLongInferredFlag'] = 1

In [28]:
(cambs_df.shape[0]-cambs_df[cambs_df['HouseNumber'] == 'Location not found'].shape[0])/cambs_df.shape[0]

0.9742813918305597

In [31]:
cambs_df.to_csv(r'C:\git\fire\area\LocationsAfterGeocoding\CambridgeshireGeocoded.csv', index=False)

### Cornwall Geocoding

In [35]:
corn_query = 'select * from initial.Cornwall'

In [36]:
corn_df = pd.read_sql(corn_query, cnxn)

In [44]:
corn_df['location'] = corn_df['location'].fillna('')
corn_df['location'] = corn_df['location'].map(lambda x: x.replace('near', ''))

In [46]:
# Geocoding using get_full_address function run on the cleaned location column

corn_df['AddressDict'] = corn_df['location'].map(lambda x: get_full_address(x))

corn_df['HouseNumber'] = corn_df['AddressDict'].map(lambda x: x['HouseNumber'])
corn_df['Road'] = corn_df['AddressDict'].map(lambda x: x['Road'])
corn_df['Town'] = corn_df['AddressDict'].map(lambda x: x['Town'])
corn_df['Postcode'] = corn_df['AddressDict'].map(lambda x: x['Postcode'])
corn_df['Lat'] = corn_df['AddressDict'].map(lambda x: x['Lat'])
corn_df['Long'] = corn_df['AddressDict'].map(lambda x: x['Long'])

In [48]:
(corn_df.shape[0]-corn_df[corn_df['HouseNumber'] == 'Location not found'].shape[0])/corn_df.shape[0]

0.6961248954558127

In [51]:
# No postcodes or LatLongs were in the original data so all flagged as inferred
corn_df['PostcodeInferredFlag'] = 1
corn_df['LatLongInferredFlag'] = 1

In [53]:
corn_df.to_csv(r'C:\git\fire\area\LocationsAfterGeocoding\CornwallGeocoded.csv', index=False)

### Cumbria Geocoding

In [54]:
cumb_query = 'select * from initial.Cumbria'

In [74]:
cumb_df = pd.read_sql(cumb_query, cnxn)

In [75]:
cumb_df['location'] = cumb_df['location'].map(lambda x: x[1:-1] + ', Cumbria')


In [77]:
cumb_df['AddressDict'] = cumb_df['location'].map(lambda x: get_full_address(x))

cumb_df['HouseNumber'] = cumb_df['AddressDict'].map(lambda x: x['HouseNumber'])
cumb_df['Road'] = cumb_df['AddressDict'].map(lambda x: x['Road'])
cumb_df['Town'] = cumb_df['AddressDict'].map(lambda x: x['Town'])
cumb_df['Postcode'] = cumb_df['AddressDict'].map(lambda x: x['Postcode'])
cumb_df['Lat'] = cumb_df['AddressDict'].map(lambda x: x['Lat'])
cumb_df['Long'] = cumb_df['AddressDict'].map(lambda x: x['Long'])

In [78]:
(cumb_df.shape[0]-cumb_df[cumb_df['HouseNumber'] == 'Location not found'].shape[0])/cumb_df.shape[0]

1.0

In [79]:
# No postcodes or LatLongs were in the original data so all flagged as inferred
cumb_df['PostcodeInferredFlag'] = 1
cumb_df['LatLongInferredFlag'] = 1

In [81]:
cumb_df.to_csv(r'C:\git\fire\area\LocationsAfterGeocoding\CumbriaGeocoded.csv', index=False)

### Devon Geocoding

In [104]:
dev_query = 'select * from [initial].[Devon]'

In [105]:
dev_df = pd.read_sql(dev_query, cnxn)

In [106]:
dev_df

,IncidentID,id,Title,Date,Time,Location,Area,County,Text
0,1,10000,Incident - Shed fire,09/07/2009,12:30,"Church Street, Morchard Bishop near Crediton.",Central,Devon,Several calls were received in the Devon Contr...
1,2,10011,Incident - Fire in Factory,10/07/2009,10:48,"Industrial Factory, Heron Road,Sowton Industri...",Central,Devon,Four fire appliances and the Aerial from Exete...
2,3,10015,Incident - Chimney Fire,10/07/2009,22:42,"Broadbury, Okehampton",Central,Devon,One fire engine from Okehampton attended an ad...
3,4,10023,Incident - Kitchen fire,11/07/2009,09:10,"St Thomas Close, Plympton",Western,Devon,Two appliances from Plympton and Crownhill wer...
4,5,10028,Incident - Dining room fire,11/07/2009,16:08,"Upland Drive, Plymouth",Western,Devon,Two appliances from Camels Head were mobilised...
5,6,10032,Incident - Shed / Green house fire,12/07/2009,05:31,"North Street, Crewkerne",Somerset,Somerset,At 0531hrs the Fire Service attended reports o...
6,7,10034,Incident - Building fire,12/07/2009,14:43,"Three Cooks, Courtenay Street, Newton Abbot",Western,Devon,Two appliances from Newton Abbot were mobilise...
7,8,10035,Incident - Kitchen Fire,12/07/2009,15:09,"Hill Crest Road, Yeovil",Somerset,Somerset,The Fire Service were called reports of a kitc...
8,9,10036,Incident - Kitchen fire,12/07/2009,15:44,"Pentamar Street, Plymouth",Western,Devon,Two appliances from Crownhill and Camels Head ...
9,10,10041,Incident - Bin Shed Fire,12/07/2009,23:06,"Wynford Road, Stoke Hill, Exeter",Central,Devon,"One fire appliance from Exeter, Watch Commande..."


In [ ]:
# not run yet, will take ~4 hours
dev_df['AddressDict'] = dev_df['Location'].map(lambda x: get_full_address(x))

dev_df['HouseNumber'] = dev_df['AddressDict'].map(lambda x: x['HouseNumber'])
dev_df['Road'] = dev_df['AddressDict'].map(lambda x: x['Road'])
dev_df['Town'] = dev_df['AddressDict'].map(lambda x: x['Town'])
dev_df['Postcode'] = dev_df['AddressDict'].map(lambda x: x['Postcode'])
dev_df['Lat'] = dev_df['AddressDict'].map(lambda x: x['Lat'])
dev_df['Long'] = dev_df['AddressDict'].map(lambda x: x['Long'])

In [ ]:
(dev_df.shape[0]-dev_df[dev_df['HouseNumber'] == 'Location not found'].shape[0])/dev_df.shape[0]

In [ ]:
# No postcodes or LatLongs were in the original data so all flagged as inferred
dev_df['PostcodeInferredFlag'] = 1
dev_df['LatLongInferredFlag'] = 1

In [ ]:
dev_df.to_csv(r'C:\git\fire\area\LocationsAfterGeocoding\DevonGeocoded.csv', index=False)

### Essex Geocoding

In [92]:
essex_query = 'select * from [initial].[Essex]'

In [93]:
essex_df = pd.read_sql(essex_query, cnxn)

In [99]:
essex_df['AddressDict'] = essex_df['Location'].map(lambda x: get_full_address(x))

essex_df['HouseNumber'] = essex_df['AddressDict'].map(lambda x: x['HouseNumber'])
essex_df['Road'] = essex_df['AddressDict'].map(lambda x: x['Road'])
essex_df['Town'] = essex_df['AddressDict'].map(lambda x: x['Town'])
essex_df['Postcode'] = essex_df['AddressDict'].map(lambda x: x['Postcode'])
essex_df['Lat'] = essex_df['AddressDict'].map(lambda x: x['Lat'])
essex_df['Long'] = essex_df['AddressDict'].map(lambda x: x['Long'])

In [110]:
(essex_df.shape[0]-essex_df[essex_df['HouseNumber'] == 'Location not found'].shape[0])/essex_df.shape[0]

0.9382504288164666

In [203]:
# No postcodes or LatLongs were in the original data so all flagged as inferred
essex_df['PostcodeInferredFlag'] = 1
essex_df['LatLongInferredFlag'] = 1

In [111]:
essex_df.to_csv(r'C:\git\fire\area\LocationsAfterGeocoding\EssexGeocoded.csv', index=False)

### Hampshire Geocoding

In [ ]:
# This isn't performing as well as it should -  sometimes when there's a postcode in the address it's confusing OSM - maybe won't happen with OpenCage

In [180]:
hamp_query = 'select * from [initial].[Hampshire]'

In [146]:
hamp_df = pd.read_sql(hamp_query, cnxn)

In [147]:
hamp_df['Address'] = hamp_df['Address'].fillna('None')
hamp_df['Address'] = hamp_df['Address'].map(lambda x: x.replace('<div class="addressDisplay">', '').replace('</div>', '').replace('---', ' ').strip())

In [149]:
hamp_df['AddressDict'] = hamp_df['Address'].map(lambda x: get_full_address(x))

hamp_df['HouseNumber'] = hamp_df['AddressDict'].map(lambda x: x['HouseNumber'])
hamp_df['Road'] = hamp_df['AddressDict'].map(lambda x: x['Road'])
hamp_df['Town'] = hamp_df['AddressDict'].map(lambda x: x['Town'])
hamp_df['Postcode'] = hamp_df['AddressDict'].map(lambda x: x['Postcode'])
hamp_df['Lat'] = hamp_df['AddressDict'].map(lambda x: x['Lat'])
hamp_df['Long'] = hamp_df['AddressDict'].map(lambda x: x['Long'])

In [151]:
(hamp_df.shape[0]-hamp_df[hamp_df['HouseNumber'] == 'Location not found'].shape[0])/hamp_df.shape[0]

0.6346820809248555

In [209]:
hamp_df['PostcodeInferredFlag'] = 0
hamp_df['PostcodeInferredFlag'] = 0
hamp_df.loc[hamp_df['Postcode'] in hamp_df['Address'], essex_df['PostcodeInferredFlag']] = 1


TypeError: 'Series' objects are mutable, thus they cannot be hashed

In [213]:
hamp_df.iloc[7]['Postcode'] in hamp_df.iloc[7]['Address']

True

In [ ]:
df.loc[df['c1'] == 'Value', 'c2']

In [210]:
hamp_df

,IncidentID,Post Date,Title,Incident Text,Address,AddressDict,HouseNumber,Road,Town,Postcode,Lat,Long,PostcodeInferredFlag
0,1,01/01/2014 21:21,Fire,"Around 38 firefighters from Hightown, Botley, ...",Botley Road West End Hampshire SO30 United Kin...,"{'HouseNumber': 'Location not found', 'Road': ...",Location not found,Location not found,Location not found,Location not found,Location not found,Location not found,0
1,2,01/04/2014 21:55,Fire,Firefighters have this evening dealt with a sm...,Commercial Road Totton Hampshire SO40 United K...,"{'HouseNumber': 'Location not found', 'Road': ...",Location not found,Location not found,Location not found,Location not found,Location not found,Location not found,0
2,3,01/04/2016 12:07,Fire - Hayling Island,"Firefighters from Hayling Island, Havant, Emsw...",Southwood Road Hayling Island PO11 United Kingdom,"{'HouseNumber': 'Location not found', 'Road': ...",Location not found,Location not found,Location not found,Location not found,Location not found,Location not found,0
3,4,01/04/2018 12:54,Crews tackle bin chute fire at Portsmouth high...,"Crews from Southsea, Cosham, Portchester, Fare...","Brownlow Close Buckland, Portsmouth Portsmouth...","{'HouseNumber': None, 'Road': 'Brownlow Close'...",None,Brownlow Close,Somers Town,PO1 4EP,50.8055,-1.08463,0
4,5,01/06/2016 22:51,Fire Domestic - Havant,Two crews from Havant and one from Cosham atte...,Soberton Road Havant Hampshire PO9 2EE United ...,"{'HouseNumber': None, 'Road': 'Soberton Road',...",None,Soberton Road,Havant,PO9 2EG,50.8633,-0.985928,0
5,6,01/07/2014 12:40,Fire,Around 14 firefighters attended a fire in a gr...,Shirley High Street Southampton Southampton Un...,"{'HouseNumber': None, 'Road': 'Shirley High St...",None,Shirley High Street,Shirley,SO15 3NP,50.9222,-1.43333,0
6,7,01/07/2014 19:17,Fire,9 Firefighters from Southsea fire station exti...,Paignton Avenue Portsmouth Portsmouth PO3 Unit...,"{'HouseNumber': None, 'Road': 'Paignton Avenue...",None,Paignton Avenue,Baffins,PO3 6LL,50.8128,-1.05982,0
7,8,01/07/2017 15:30,"Gym, workshop and garden centre damaged in Whi...",About 30 firefighters battled a blaze after a ...,Church Street Whitchurch Hampshire RG28 7AD Un...,"{'HouseNumber': None, 'Road': 'Church Street',...",None,Church Street,Whitchurch,RG28 7AD,51.2294,-1.33968,0
8,9,01/07/2018 03:35,Cigarette causes Southampton balcony fire,"Five engines, an aerial ladder platform and su...",Ocean Village Southampton Southampton SO14 3JE...,"{'HouseNumber': None, 'Road': None, 'Town': 'O...",None,None,Ocean Village,None,50.8956,-1.39204,0
9,10,01/08/2015 18:15,Fire - Beaulieu,Around six firefighters from Lyndhurst extingu...,Beaulieu Road SO42 United Kingdom,"{'HouseNumber': None, 'Road': 'Beaulieu Road',...",None,Beaulieu Road,Denny Lodge,SO42 7YQ,50.863,-1.528,0


### Kent Geocoding

In [181]:
kent_query = 'select * from [initial].[Kent]'

In [182]:
kent_df = pd.read_sql(kent_query, cnxn)

In [ ]:
# not run yet, will take ~45 minutes - incorrect postcodes are likely to mess up OSM 
kent_df['AddressDict'] = kent_df['Map - location shows general area of incident'].map(lambda x: get_full_address(x))

kent_df['HouseNumber'] = kent_df['AddressDict'].map(lambda x: x['HouseNumber'])
kent_df['Road'] = kent_df['AddressDict'].map(lambda x: x['Road'])
kent_df['Town'] = kent_df['AddressDict'].map(lambda x: x['Town'])
kent_df['Postcode'] = kent_df['AddressDict'].map(lambda x: x['Postcode'])
kent_df['Lat'] = kent_df['AddressDict'].map(lambda x: x['Lat'])
kent_df['Long'] = kent_df['AddressDict'].map(lambda x: x['Long'])

In [ ]:
(kent_df.shape[0]-kent_df[kent_df['HouseNumber'] == 'Location not found'].shape[0])/kent_df.shape[0]

In [221]:
get_full_address('Frensham, Dockenfield and Tilford, Waverley')

{'HouseNumber': None,
 'Road': 'The Reeds Road',
 'Town': 'Tilford',
 'Postcode': 'GU10 3BP',
 'Lat': 51.1784402,
 'Long': -0.778476959685393}